In [1]:
import os
import pandas as pd
import anndata as ad

In [2]:
meta = pd.read_csv("../../1_NanoNASCseq/reports/NanoNASCseq_Summary.csv", index_col=1)
meta = meta[meta["Group"] == "MouseBlastocyst"]
meta = meta[meta["UMIs"] >= 5000]
print("Cells:", len(meta))

Cells: 2661


In [3]:
path1 = "results/blastocyst_counts.gene_based.filelist.txt"
with open(path1, "w+") as fw:
    for cell in meta.index:
        fw.write("../../1_NanoNASCseq/results/expression/quant_genes/min_read_2_min_tc_2/%s/%s.tsv" % (cell.split(".")[0], cell) + "\n")

In [4]:
path2 = "results/blastocyst_counts.gene_based.gene_id.h5ad"
if not os.path.exists(path2):
    ! ../../1_NanoNASCseq/scripts/make_matrix.py {path1} {path2}

In [5]:
path3 = "results/blastocyst_counts.gene_based.gene_id.annotated.h5ad"
anno = pd.read_csv("../../1_NanoNASCseq/results/assembly_custom/gtf_full/MouseBlastocyst.gtf.gene_info.tsv", sep="\t", index_col=0)
adata = ad.read_h5ad(path2)
adata.obs = adata.obs.merge(meta, left_index=True, right_index=True, how="left")
adata.obs["CellLine"] = ""
adata.var = adata.var.merge(anno, left_index=True, right_index=True, how="left")
adata.write(path3, compression="gzip")
adata

AnnData object with n_obs × n_vars = 2661 × 29073
    obs: 'Run', 'Barcode', 'Species', 's4U', 'Time', 'ActD', 'Group', 'CellLine', 'Stage', 'Platform', 'Label', 'UMI', 'Cells', 'LibStruct', 'Remark', 'Cell.Reads', 'Trimmed.Reads', 'Trimmed.Ratio', 'Mapped.Reads', 'Mapped.Ratio', 'Mito.Ratio', 'Filtered.Reads', 'Filtered.Ratio', 'FilteredClip.Reads', 'FilteredClip.Ratio', 'UMIs', 'UMIs.2Reads', 'Duplicate.Reads', 'Duplicate.Ratio', 'Unique.Reads', 'Genes', 'Isoforms.Assembled', 'Isoforms.Known', 'AC.Ratio', 'AG.Ratio', 'AT.Ratio', 'CA.Ratio', 'CG.Ratio', 'CT.Ratio', 'GA.Ratio', 'GC.Ratio', 'GT.Ratio', 'TA.Ratio', 'TC.Ratio', 'TG.Ratio', 'Pe', 'Pc', 'PcPe.Ratio', 'UMIs.2Reads.Nascent.2TCs', 'UMIs.2Reads.Nascent.2TCs.Ratio', 'Genes.2Reads', 'Genes.2Reads.Nascent.2TCs'
    var: 'GeneName', 'GeneType', 'Chrom', 'Start', 'End', 'Strand'
    layers: 'nascent', 'total'

In [6]:
path4 = "results/blastocyst_counts.gene_based.gene_name.annotated.h5ad"
adata = adata[:,~adata.var["GeneName"].isna()] 
adata = adata[:,~adata.var["GeneName"].duplicated()]
adata.var.insert(0, "GeneID", adata.var.index)
adata.var.index = adata.var["GeneName"]
del adata.var["GeneName"]
adata.write(path4, compression="gzip")
adata

View of AnnData object with n_obs × n_vars = 2661 × 28794
    obs: 'Run', 'Barcode', 'Species', 's4U', 'Time', 'ActD', 'Group', 'CellLine', 'Stage', 'Platform', 'Label', 'UMI', 'Cells', 'LibStruct', 'Remark', 'Cell.Reads', 'Trimmed.Reads', 'Trimmed.Ratio', 'Mapped.Reads', 'Mapped.Ratio', 'Mito.Ratio', 'Filtered.Reads', 'Filtered.Ratio', 'FilteredClip.Reads', 'FilteredClip.Ratio', 'UMIs', 'UMIs.2Reads', 'Duplicate.Reads', 'Duplicate.Ratio', 'Unique.Reads', 'Genes', 'Isoforms.Assembled', 'Isoforms.Known', 'AC.Ratio', 'AG.Ratio', 'AT.Ratio', 'CA.Ratio', 'CG.Ratio', 'CT.Ratio', 'GA.Ratio', 'GC.Ratio', 'GT.Ratio', 'TA.Ratio', 'TC.Ratio', 'TG.Ratio', 'Pe', 'Pc', 'PcPe.Ratio', 'UMIs.2Reads.Nascent.2TCs', 'UMIs.2Reads.Nascent.2TCs.Ratio', 'Genes.2Reads', 'Genes.2Reads.Nascent.2TCs'
    var: 'GeneID', 'GeneType', 'Chrom', 'Start', 'End', 'Strand'
    layers: 'nascent', 'total'